<a href="https://colab.research.google.com/github/YashDeepp/RNN-Text-Generation/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm

In [2]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __len__(self):
        return len(self.word2idx)


In [3]:
class TextProcess(object):

    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
        rep_tensor = torch.LongTensor(tokens)
        index = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    rep_tensor[index] = self.dictionary.word2idx[word]
                    index += 1
        num_batches = rep_tensor.shape[0] // batch_size
        rep_tensor = rep_tensor[:num_batches*batch_size]
        rep_tensor = rep_tensor.view(batch_size, -1)
        return rep_tensor

In [19]:
embed_size = 128    #Input features to the LSTM
hidden_size = 1024  #LSTM units
num_layers = 1
num_epochs = 500
batch_size = 20
timesteps = 30 # Looks at 30 previous words to predict next word
learning_rate = 0.002

In [6]:
corpus = TextProcess()

In [8]:
rep_tensor = corpus.get_data('/content/drive/MyDrive/RNN/alice.txt', batch_size)
print(rep_tensor.shape)

torch.Size([20, 1484])


In [9]:
vocab_size = len(corpus.dictionary)
print(vocab_size)

5290


In [11]:
num_batches = rep_tensor.shape[1] // timesteps
print(num_batches)

49


In [13]:
class TextGenerator(nn.Module):

    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(TextGenerator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        x = self.embed(x)
        out, (h, c) = self.lstm(x, h)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        out = self.linear(out)
        return out, (h, c)

In [14]:
model = TextGenerator(vocab_size, embed_size, hidden_size, num_layers)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size),
              torch.zeros(num_layers, batch_size, hidden_size))

    for i in range(0, rep_tensor.size(1) - timesteps, timesteps):
        inputs = rep_tensor[:, i:i+timesteps]  #i/p --->(:,0:30) o/p (:,1:31)
        targets = rep_tensor[:, (i+1):(i+1)+timesteps]
        #Outpu is (output,(h_n,c_n))
        outputs,_ = model(inputs, states)
        loss = loss_fn(outputs, targets.reshape(-1))
        model.zero_grad()
        loss.backward()
        clip_grad_norm(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // timesteps
        if step % 100 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, loss.item()))

In [21]:
with torch.no_grad():
    with open('results.txt', 'w') as f:
        state = (torch.zeros(num_layers, 1, hidden_size),
                 torch.zeros(num_layers, 1, hidden_size))
        input = torch.randint(0,vocab_size, (1,)).long().unsqueeze(1)

        for i in range(500):
            output, _ = model(input, state)
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()
            input.fill_(word_id)

            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, 500, 'results.txt'))

Sampled [100/500] words and save to results.txt
Sampled [200/500] words and save to results.txt
Sampled [300/500] words and save to results.txt
Sampled [400/500] words and save to results.txt
Sampled [500/500] words and save to results.txt
